<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

## Data Load & Prep

In this notebook we show how to download and preprocess the [SNLI](https://nlp.stanford.edu/projects/snli/) dataset for sentence similarity.

We show how to apply common preprocessing steps (lowercase standardization and tokenization) to the  [SNLI](https://nlp.stanford.edu/projects/snli/) corpus. We also reshape the data into the form needed to implement [Gensen](https://github.com/Maluuba/gensen), a model that learns rich fixed-length sentence embeddings as described in the paper [here](https://openreview.net/forum?id=B18WgG-CZ&noteId=B18WgG-CZ).

### 00 Global Settings

In [1]:
import sys
sys.path.append("../../../")

import os
from utils_nlp.dataset.preprocess import to_lowercase, to_nltk_tokens
from utils_nlp.dataset import snli

print("System version: {}".format(sys.version))

System version: 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]


In [2]:
BASE_DATA_PATH = '../../../data'

### 01 Load SNLI as a pandas dataframe
We provide a function `load_pandas_df` which
* Downloads the SNLI zipfile at the specified directory location
* Extracts the file based on the specified split
* Loads the split as a pandas dataframe
The zipfile contains the following files:
* snli_1.0_dev.txt
* snli_1.0_train.txt
* snli_1.0_test.tx
* snli_1.0_dev.jsonl
* snli_1.0_train.jsonl
* snli_1.0_test.jsonl  
The loader defaults to reading from the .txt file; however, the user can change this to .jsonl by setting the optional `file_type` parameter when calling the function.

In [3]:
# defaults to txt
train = snli.load_pandas_df(BASE_DATA_PATH, file_split="train")

#load dataframe from jsonl file format
dev = snli.load_pandas_df(BASE_DATA_PATH, file_split="dev", file_type="jsonl")

#specify txt format 
test = snli.load_pandas_df(BASE_DATA_PATH, file_split="test", file_type="txt")


In [6]:
train.head()

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5
0,neutral,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4,3416050480.jpg#4r1n,neutral,NaN,NaN,NaN,NaN
1,contradiction,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",3416050480.jpg#4,3416050480.jpg#4r1c,contradiction,NaN,NaN,NaN,NaN
2,entailment,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,"( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",3416050480.jpg#4,3416050480.jpg#4r1e,entailment,NaN,NaN,NaN,NaN
3,neutral,( Children ( ( ( smiling and ) waving ) ( at c...,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,Children smiling and waving at camera,They are smiling at their parents,2267923837.jpg#2,2267923837.jpg#2r1n,neutral,NaN,NaN,NaN,NaN
4,entailment,( Children ( ( ( smiling and ) waving ) ( at c...,( There ( ( are children ) present ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,Children smiling and waving at camera,There are children present,2267923837.jpg#2,2267923837.jpg#2r1e,entailment,NaN,NaN,NaN,NaN


In [7]:
dev.head()

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,"[neutral, entailment, neutral, neutral, neutral]",4705552913.jpg#2,neutral,4705552913.jpg#2r1n,Two women are embracing while holding to go pa...,( ( Two women ) ( ( are ( embracing ( while ( ...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,The sisters are hugging goodbye while holding ...,( ( The sisters ) ( ( are ( ( hugging goodbye ...,(ROOT (S (NP (DT The) (NNS sisters)) (VP (VBP ...
1,"[entailment, entailment, entailment, entailmen...",4705552913.jpg#2,entailment,4705552913.jpg#2r1e,Two women are embracing while holding to go pa...,( ( Two women ) ( ( are ( embracing ( while ( ...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,Two woman are holding packages.,( ( Two woman ) ( ( are ( holding packages ) )...,(ROOT (S (NP (CD Two) (NN woman)) (VP (VBP are...
2,"[contradiction, contradiction, contradiction, ...",4705552913.jpg#2,contradiction,4705552913.jpg#2r1c,Two women are embracing while holding to go pa...,( ( Two women ) ( ( are ( embracing ( while ( ...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,The men are fighting outside a deli.,( ( The men ) ( ( are ( fighting ( outside ( a...,(ROOT (S (NP (DT The) (NNS men)) (VP (VBP are)...
3,"[entailment, entailment, entailment, entailmen...",2407214681.jpg#0,entailment,2407214681.jpg#0r1e,"Two young children in blue jerseys, one with t...",( ( ( Two ( young children ) ) ( in ( ( ( ( ( ...,(ROOT (S (NP (NP (CD Two) (JJ young) (NNS chil...,Two kids in numbered jerseys wash their hands.,( ( ( Two kids ) ( in ( numbered jerseys ) ) )...,(ROOT (S (NP (NP (CD Two) (NNS kids)) (PP (IN ...
4,"[neutral, neutral, neutral, entailment, entail...",2407214681.jpg#0,neutral,2407214681.jpg#0r1n,"Two young children in blue jerseys, one with t...",( ( ( Two ( young children ) ) ( in ( ( ( ( ( ...,(ROOT (S (NP (NP (CD Two) (JJ young) (NNS chil...,Two kids at a ballgame wash their hands.,( ( ( Two kids ) ( at ( a ballgame ) ) ) ( ( w...,(ROOT (S (NP (NP (CD Two) (NNS kids)) (PP (IN ...


In [8]:
test.head()

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5
0,neutral,( ( This ( church choir ) ) ( ( ( sings ( to (...,( ( The church ) ( ( has ( cracks ( in ( the c...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,(ROOT (S (NP (DT The) (NN church)) (VP (VBZ ha...,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,2677109430.jpg#1,2677109430.jpg#1r1n,neutral,contradiction,contradiction,neutral,neutral
1,entailment,( ( This ( church choir ) ) ( ( ( sings ( to (...,( ( The church ) ( ( is ( filled ( with song )...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,(ROOT (S (NP (DT The) (NN church)) (VP (VBZ is...,This church choir sings to the masses as they ...,The church is filled with song.,2677109430.jpg#1,2677109430.jpg#1r1e,entailment,entailment,entailment,neutral,entailment
2,contradiction,( ( This ( church choir ) ) ( ( ( sings ( to (...,( ( ( A choir ) ( singing ( at ( a ( baseball ...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,(ROOT (NP (NP (DT A) (NN choir)) (VP (VBG sing...,This church choir sings to the masses as they ...,A choir singing at a baseball game.,2677109430.jpg#1,2677109430.jpg#1r1c,contradiction,contradiction,contradiction,contradiction,contradiction
3,neutral,( ( ( A woman ) ( with ( ( ( ( ( a ( green hea...,( ( The woman ) ( ( is young ) . ) ),(ROOT (NP (NP (DT A) (NN woman)) (PP (IN with)...,(ROOT (S (NP (DT The) (NN woman)) (VP (VBZ is)...,"A woman with a green headscarf, blue shirt and...",The woman is young.,6160193920.jpg#4,6160193920.jpg#4r1n,neutral,neutral,neutral,neutral,neutral
4,entailment,( ( ( A woman ) ( with ( ( ( ( ( a ( green hea...,( ( The woman ) ( ( is ( very happy ) ) . ) ),(ROOT (NP (NP (DT A) (NN woman)) (PP (IN with)...,(ROOT (S (NP (DT The) (NN woman)) (VP (VBZ is)...,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,6160193920.jpg#4,6160193920.jpg#4r1e,entailment,entailment,contradiction,entailment,neutral


### 02 Tokenize

We have loaded the dataset into pandas.DataFrame, we now convert sentences to tokens.
We also clean the data before tokenizing. This includes dropping unneccessary columns and renaming the relevant columns as score, sentence_1, and sentence_2.

In [9]:
def clean(df, file_split):
    src_file_path = os.path.join(BASE_DATA_PATH, "raw/snli_1.0/snli_1.0_{}.txt".format(file_split))
    return snli.clean_snli(src_file_path).dropna() # drop rows with any NaN vals

In [10]:
train = clean(train, 'train')
dev = clean(dev, 'dev')
test = clean(test, 'test')

Glimpse of the data

In [11]:
train.head()

,score,sentence1,sentence2
0,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.
1,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette."
2,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse."
3,neutral,Children smiling and waving at camera,They are smiling at their parents
4,entailment,Children smiling and waving at camera,There are children present


In [12]:
dev.head()

,score,sentence1,sentence2
0,neutral,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...
1,entailment,Two women are embracing while holding to go pa...,Two woman are holding packages.
2,contradiction,Two women are embracing while holding to go pa...,The men are fighting outside a deli.
3,entailment,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.
4,neutral,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.


In [13]:
test.head()

,score,sentence1,sentence2
0,neutral,This church choir sings to the masses as they ...,The church has cracks in the ceiling.
1,entailment,This church choir sings to the masses as they ...,The church is filled with song.
2,contradiction,This church choir sings to the masses as they ...,A choir singing at a baseball game.
3,neutral,"A woman with a green headscarf, blue shirt and...",The woman is young.
4,entailment,"A woman with a green headscarf, blue shirt and...",The woman is very happy.


Once we have the clean pandas dataframes, we do lowercase standardization and tokenization. We use the [NLTK] (https://www.nltk.org/) library for tokenization.

In [14]:
train_tok = to_nltk_tokens(to_lowercase(train))
dev_tok = to_nltk_tokens(to_lowercase(dev))
test_tok = to_nltk_tokens(to_lowercase(test))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jamahaja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jamahaja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jamahaja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
dev_tok.head()

,score,sentence1,sentence2,sentence1_tokens,sentence2_tokens
0,neutral,two women are embracing while holding to go pa...,the sisters are hugging goodbye while holding ...,"[two, women, are, embracing, while, holding, t...","[the, sisters, are, hugging, goodbye, while, h..."
1,entailment,two women are embracing while holding to go pa...,two woman are holding packages.,"[two, women, are, embracing, while, holding, t...","[two, woman, are, holding, packages, .]"
2,contradiction,two women are embracing while holding to go pa...,the men are fighting outside a deli.,"[two, women, are, embracing, while, holding, t...","[the, men, are, fighting, outside, a, deli, .]"
3,entailment,"two young children in blue jerseys, one with t...",two kids in numbered jerseys wash their hands.,"[two, young, children, in, blue, jerseys, ,, o...","[two, kids, in, numbered, jerseys, wash, their..."
4,neutral,"two young children in blue jerseys, one with t...",two kids at a ballgame wash their hands.,"[two, young, children, in, blue, jerseys, ,, o...","[two, kids, at, a, ballgame, wash, their, hand..."


### 03 Reshape for GenSen model
We need to prepare our data in a specific way in order for the Gensen model to be able to ingest it. We do this by
* Saving the tokens for each split in a `snli_1.0_{split}.txt.clean` file, with the sentence pairs and scores tab-separated and the tokens separated by a single space.
* Saving the tokenized sentence and labels separately, in the form `snli_1.0_{split}.txt.s1.tok` or `snli_1.0_{split}.txt.s2.tok` or `snli_1.0_{split}.txt.lab`.

In [16]:
split_map = {'train': train_tok, 'dev': dev_tok, 'test': test_tok}
for file_split, df in split_map.items():
    base_txt_path = os.path.join(BASE_DATA_PATH, "clean/snli_1.0/snli_1.0_{}.txt".format(file_split))
    df['s1.tok'] = df['sentence1_tokens'].apply(lambda x: ' '.join(x))
    df['s2.tok'] = df['sentence2_tokens'].apply(lambda x: ' '.join(x))
    df['s1.tok'].to_csv("{}.s1.tok".format(base_txt_path), sep=' ', header=False, index=False)
    df['s2.tok'].to_csv("{}.s2.tok".format(base_txt_path), sep=' ', header=False, index=False)
    df['score'].to_csv("{}.lab".format(base_txt_path), sep=' ', header=False, index=False)
    df[['s1.tok', 's2.tok', 'score']].to_csv("{}.clean".format(base_txt_path), sep="\t", header=False, index=False)

In [17]:
# remove quotations from .tok files
import shutil

for file_split in split_map.keys():
    s1_tok_path = os.path.join(BASE_DATA_PATH, "clean/snli_1.0/snli_1.0_{}.txt.s1.tok".format(file_split))
    s2_tok_path = os.path.join(BASE_DATA_PATH, "clean/snli_1.0/snli_1.0_{}.txt.s2.tok".format(file_split))
    with open(s1_tok_path, 'r') as fin, open("{}.tmp".format(s1_tok_path), 'w') as tmp:
        for line in fin:
            s = line.replace('\"', '')
            tmp.write(s)
    with open(s2_tok_path, 'r') as fin, open("{}.tmp".format(s2_tok_path), 'w') as tmp:
        for line in fin:
            s = line.replace('\"', '')
            tmp.write(s)
    shutil.move("{}.tmp".format(s1_tok_path), s1_tok_path)
    shutil.move("{}.tmp".format(s2_tok_path), s2_tok_path)